In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from splinter import Browser
import warnings
warnings.filterwarnings('ignore')
print('Libraries imported!')

Libraries imported!


#url = 'https://geocoder.ca/?locate='
#postalCode = 'T2L+0C9'/
#page = requests.get(url+postalCode)
#soup = BeautifulSoup(page.text, "html.parser")
#print(soup.prettify())

tables = pd.read_html(url+postalCode)
table_df = tables[1]
table_df.head()

title = soup.find('li', class_='pull-right').get_text()
print(title)

In [2]:
csvData = pd.read_csv('data/score_df.csv')
csvData.head()

,postal_code,walk_score,bike_score,transit_score
0,T2G 0G3,91.0,92.0,77.0
1,T3M 0A8,36.0,63.0,40.0
2,T2T 3E9,90.0,82.0,66.0
3,T2J 7H1,53.0,68.0,32.0
4,T3L 1Y4,39.0,61.0,41.0


In [3]:
csvDataPC = csvData['postal_code']
csvDataPC.head()
#testCSV = csvDataPC.head(20)

0    T2G 0G3
1    T3M 0A8
2    T2T 3E9
3    T2J 7H1
4    T3L 1Y4
Name: postal_code, dtype: object

In [4]:
len(csvDataPC)

2270

csvDataPC = csvDataPC.str.replace(' ', '+')
testCSV = csvDataPC.head(20)

coordinates = []
postalCodes = []
for i in csvDataPC:
    url = 'https://geocoder.ca/?locate='
    page = requests.get(url+str(i))
    time.sleep(5)
    soup = BeautifulSoup(page.text, "html.parser")
    try:
        title = soup.find('li', class_='pull-right').get_text()
    except:
        title = 'None'
    coordinates.append(title)
    #postalCodesP = i.str.replace('+', ' ')
    postalCodes.append(i)

coordinatesDF = pd.DataFrame(coordinates, postalCodes)
coordinatesDF.reset_index(inplace=True)

coordinatesDF.columns=['postal_codes', 'coordinates']
coordinatesDF.head()

In [6]:
import pgeocode

In [7]:
coordinates = []
postalCodes = []
for i in csvDataPC:
    country_code = pgeocode.Nominatim('ca')
    coord = country_code.query_postal_code(str(i))
    x = [coord.latitude, coord.longitude]
    coordinates.append(x)
    postalCodes.append(str(i))

In [10]:
len(coordinates)

2270

In [11]:
coordinatesDF = pd.DataFrame(coordinates, postalCodes)
coordinatesDF.reset_index(inplace=True)

In [12]:
coordinatesDF.head()

,index,0,1
0,T2G 0G3,51.0272,-114.0349
1,T3M 0A8,50.8796,-113.9555
2,T2T 3E9,51.0242,-114.1004
3,T2J 7H1,50.8476,-114.1958
4,T3L 1Y4,51.1467,-114.3133


In [13]:
coordinatesDF.columns=['postal_codes', 'lat', 'long']
coordinatesDF.head()

,postal_codes,lat,long
0,T2G 0G3,51.0272,-114.0349
1,T3M 0A8,50.8796,-113.9555
2,T2T 3E9,51.0242,-114.1004
3,T2J 7H1,50.8476,-114.1958
4,T3L 1Y4,51.1467,-114.3133


In [14]:
coordinatesDF.to_csv('data/cal_coordinatesDF.csv', index=False)

In [15]:
import pymongo
from pymongo import MongoClient

conn = 'mongodb://localhost:27017'

## Making a Connection with MongoClient
client = MongoClient(conn)
# database
db = client.realestate_db


collection = db.coordinates
coordinates_dict = coordinatesDF.to_dict("records")
collection.insert_many(coordinates_dict)

In [18]:
from sqlalchemy import create_engine

In [19]:
rds_connection_string = 'postgres:1@localhost:5432/realestate_db'
engine = create_engine(f'postgresql://{rds_connection_string}')
coordinatesDF.to_sql(name= 'coordinates_df', con=engine, if_exists = 'replace', index=False)